In [1]:
#import packages
#pip install textblob
#pip install keras
#pip install tensorflow
import time
from textblob import TextBlob
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import numpy as np

#for CNN 
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

2023-04-23 19:02:03.265105: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
year = "2022"
file_path = "/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/"
file_path_2 = "_final_rnn.pickle"

df_22= pd.read_pickle(file_path + year + file_path_2)

In [3]:
def absolute_count(male_col, female_col):
    if female_col > male_col and male_col == 0:
        return 1
    elif male_col> female_col and female_col ==0: 
        return 0
    else: 
        return None

#PP Data
df_22['col_type'] = df_22.apply(lambda row: absolute_count(row['male_count'], row['female_count']),axis=1)
df_22= df_22[df_22["col_type"].notnull()]
df_22

,pre_processed_sent,string_rnn,male_count,female_count,Proper_noun_list,pn exists,sentences,article_id,year,col_type
0,"[add, three, decade, building, extremely, prou...",add three decade building extremely proud grou...,1,0,"[Balhousie, Care]",True,"He added: ""After three decades of building Bal...",13,2022,0.0
1,"[hospitalise, number, facial, fracture, follow...",hospitalise number facial fracture follow seri...,2,0,[],None,A man in his 20s has been hospitalised with a ...,24,2022,0.0
2,"[victim, front, passenger, seat, vehicle, atta...",victim front passenger seat vehicle attack ano...,5,0,[],None,The victim was in the front passenger seat of ...,24,2022,0.0
3,"[involve, bitter, dispute, organisation, membe...",involve bitter dispute organisation member sta...,0,1,[Baroness],None,"Baroness Scotland, Secretary General of the Co...",30,2022,1.0
4,"[add, chair, regret, challenge, position, take...",add chair regret challenge position take secre...,1,0,[],None,"""He adds: ""The chair regrets and challenges th...",30,2022,0.0
...,...,...,...,...,...,...,...,...,...,...
410780,"[ground, hard, thaw, yet, come, experience, sh...",ground hard thaw yet come experience show u th...,1,0,[],None,"""The ground has been very hard, the thaw is ye...",1043999,2022,0.0
410781,"[endeavour, get, ahead, game, best, find, situ...",endeavour get ahead game best find situation c...,1,0,[],None,"""What we are endeavouring to do here is get ah...",1043999,2022,0.0
410782,"[die, hit, garage, forecourt]",die hit garage forecourt,1,0,[],None,A man has died after being hit by a 4x4 on a g...,1044006,2022,0.0
410783,"[north, say, incident, garage, happen, area, t...",north say incident garage happen area take hos...,2,0,"[Wales, Chester]",True,North Wales Police said the incident at the Pr...,1044006,2022,0.0


In [15]:
def subjectivity(sentence):
    subjectivity = ""

    subjectivity = TextBlob(sentence).sentiment.subjectivity

    return subjectivity

def polarity(sentence):
    polarity = ""
    polarity = TextBlob(sentence).sentiment.polarity
    return polarity

In [16]:
df_22['subjectivity'] = df_22['string_rnn'].apply(subjectivity)
df_22['polarity'] = df_22['string_rnn'].apply(polarity)

In [17]:
df_22

,pre_processed_sent,string_rnn,male_count,female_count,Proper_noun_list,pn exists,sentences,article_id,year,col_type,subjectivity,polarity
0,"[add, three, decade, building, extremely, prou...",add three decade building extremely proud grou...,1,0,"[Balhousie, Care]",True,"He added: ""After three decades of building Bal...",13,2022,0,0.770000,0.480000
1,"[hospitalise, number, facial, fracture, follow...",hospitalise number facial fracture follow seri...,2,0,[],None,A man in his 20s has been hospitalised with a ...,24,2022,0,0.333333,-0.166667
2,"[victim, front, passenger, seat, vehicle, atta...",victim front passenger seat vehicle attack ano...,5,0,[],None,The victim was in the front passenger seat of ...,24,2022,0,0.050000,-0.075000
3,"[involve, bitter, dispute, organisation, membe...",involve bitter dispute organisation member sta...,0,1,[Baroness],None,"Baroness Scotland, Secretary General of the Co...",30,2022,1,0.250000,-0.050000
4,"[add, chair, regret, challenge, position, take...",add chair regret challenge position take secre...,1,0,[],None,"""He adds: ""The chair regrets and challenges th...",30,2022,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
410780,"[ground, hard, thaw, yet, come, experience, sh...",ground hard thaw yet come experience show u th...,1,0,[],None,"""The ground has been very hard, the thaw is ye...",1043999,2022,0,0.413889,-0.230556
410781,"[endeavour, get, ahead, game, best, find, situ...",endeavour get ahead game best find situation c...,1,0,[],None,"""What we are endeavouring to do here is get ah...",1043999,2022,0,0.350000,0.300000
410782,"[die, hit, garage, forecourt]",die hit garage forecourt,1,0,[],None,A man has died after being hit by a 4x4 on a g...,1044006,2022,0,0.000000,0.000000
410783,"[north, say, incident, garage, happen, area, t...",north say incident garage happen area take hos...,2,0,"[Wales, Chester]",True,North Wales Police said the incident at the Pr...,1044006,2022,0,0.000000,0.000000


In [21]:
len(df_22)

410785

In [30]:
def scores(df, col): 
    sub = []
    pol = []
    count = 0
    for index, row in df.iterrows():
        if row[col] > 1: 
            sub.append(row["subjectivity"])
            pol.append(row["polarity"])
        count +=1
    sub = [s/count for s in sub]
    pol = [p/count for p in pol]
    print(sub)
    print(pol)
        

In [31]:
scores(df_22, 'male_count')

[8.114544916034746e-07, 1.217181737405212e-07, 1.3910648427488136e-06, 0.0, 0.0, 1.217181737405212e-06, 0.0, 2.1909271273293818e-06, 0.0, 1.217181737405212e-07, 0.0, 0.0, 2.434363474810424e-07, 1.2222533279777337e-06, 0.0, 0.0, 7.384235873591618e-07, 9.585306182066044e-07, 7.100226801530402e-07, 3.955840646566939e-07, 1.217181737405212e-06, 0.0, 1.217181737405212e-06, 8.925999407638222e-07, 7.370711632064894e-07, 1.2678976431304289e-06, 8.288428021378349e-07, 0.0, 1.1451651512754034e-06, 3.04295434351303e-07, 0.0, 7.475217134771403e-07, 4.868726949620848e-07, 0.0, 3.955840646566939e-07, 0.0, 0.0, 1.217181737405212e-06, 4.868726949620848e-07, 8.114544916034746e-07, 1.8744598756040264e-06, 1.217181737405212e-06, 1.054890839084517e-06, 2.434363474810424e-06, 1.3794726357259068e-06, 1.947490779848339e-06, 1.0954635636646907e-06, 7.332070941988538e-07, 2.434363474810424e-07, 9.737453899241696e-07, 0.0, 0.0, 2.434363474810424e-07, 1.5958605001535e-06, 1.3693294545808635e-06, 1.23746809969529

In [ ]:
#create a 2 point criteria from 0 to 1 (range of subjectivity)
def map_sentiment(value):
    if value >= 0.5: 
        return 1
    elif value <= -0.33:
        return -1
    else:
        return 0


df_09['sentiment'] = df_09['polarity'].apply(map_sentiment)
df_09['sentiment'] = df_09["sentiment"].astype(float)
df_09["sentiment"].value_counts()

**RNN LSTM Model for Sentiment Analysis** 

This classifier identifies the polarity of the sentences based on the subjectivity. It will be used to cross-reference sentiment along with the male or female nature of the columns. 

In [ ]:
#BEST LSTM FUNCTION I COULD FINETUNE
def rnn_lstm(df, sentences_col, sentiment_col, gender_col):
    # Start timer 
    start_time = time.time()

    X = df[sentences_col]
    y = df[sentiment_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    #gender col 
    gender_col = df[gender_col]
    
    # Tokenize the data
    tokenizer = Tokenizer(num_words=1000)
    tokenizer.fit_on_texts(X_train)
    
    # Convert the texts to sequences
    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_test_seq = tokenizer.texts_to_sequences(X_test)
    
    # Pad the sequences to ensure equal length
    maxlen = 100
    X_train_padded = pad_sequences(X_train_seq, padding='post', maxlen=maxlen)
    X_test_padded = pad_sequences(X_test_seq, padding='post', maxlen=maxlen)
    
    # Define the LSTM model
    model_lstm = Sequential()
    model_lstm.add(Embedding(input_dim=1000, output_dim=64, input_length=maxlen))
    model_lstm.add(LSTM(64))
    model_lstm.add(Dense(1, activation='sigmoid'))
    model_lstm.compile(loss='binary_crossentropy', optimizer='RMSProp', metrics=['accuracy', "mean_squared_error"])
    
    # Train the model
    model_lstm.fit(X_train_padded, y_train, epochs=3, batch_size=32, validation_data=(X_test_padded, y_test))
    
    # Evaluate the model
    score = model_lstm.evaluate(X_test_padded, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    # Add early stopping 
    earlystop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

    # Make predictions on the padded sequences
    y_pred = model_lstm.predict(X_test_padded)
    
    # Convert sequences back to strings for TextBlob analysis
    X_train_str = [' '.join([tokenizer.index_word.get(word) for word in sent]) for sent in X_train_seq]
    X_test_str = [' '.join([tokenizer.index_word.get(word) for word in sent]) for sent in X_test_seq]
    
    # Calculate the sentiment polarity of each sentence using TextBlob
    X_train_polarity = [TextBlob(sent).sentiment.polarity for sent in X_train_str]
    X_test_polarity = [TextBlob(sent).sentiment.polarity for sent in X_test_str]
    
    # Convert polarity values to sentiment labels
    y_train_sentiment = ['positive' if polarity > 0.33 else 'negative' if polarity <- 0.33 else 'neutral' for polarity in X_train_polarity]
    y_test_sentiment = ['positive' if polarity > 0.33 else'negative' if polarity <- 0.33 else 'neutral' for polarity in X_test_polarity]
                        

    # Create a DataFrame of the test data, predicted sentiment, and gender
    df_results = pd.DataFrame({'Sentence': X_test, 'Sentiment': y_test_sentiment, 'Gender': gender_col[X_test.index]}) #,'Predicted Sentiment': y_test_sentiment.flatten()})
    
    # End timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    return df_results


In [ ]:
senti_df09 = rnn_lstm(df_09, "sentences", "sentiment", "col_type")
senti_df09

**CNN Model for Sentiment Analysis (WE SHOULD USE THIS)**

This CNN loads a dataset of sentences within documents labeled as positive or negative, tokenizes the text, and uses a CNN to classify the sentiment of the text. It trains the model, evaluates it on a test set, and makes predictions. 

In [ ]:
#CNN using rule-bases (Textblob approach)
def cnn(df):
    # Start timer 
    start_time = time.time()
    
    #virgin code CNN
    X = df['sentences']
    y = df['sentiment']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the data
    tokenizer = Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(X_train)

# Convert the texts to sequences
    X_train = tokenizer.texts_to_sequences(X_train)
    X_test = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to ensure equal length
    maxlen = 100
    X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
    X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Define the CNN model
    model = Sequential()
    model.add(Embedding(input_dim=5000, output_dim=128, input_length=maxlen))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', "mean_squared_error"])

# Train the model
    model.fit(X_train, y_train, epochs=3, batch_size=64, validation_data=(X_test, y_test))

    y_pred = model.predict(X_test)

# Evaluate the model
    score = model.evaluate(X_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    #print(type(X_test))

    X_test = tokenizer.sequences_to_texts(X_test)


# Get the corresponding gender value
    X_t = pd.DataFrame(X_test)
    gender = df.iloc[X_t.index]['col_type']
    #print(type(gender))
    y_p = pd.DataFrame(y_pred, columns = ["Senti"])
    g = pd.DataFrame(gender)

    #save model 
    #with open('results_sentiment.pkl', 'wb') as f:
        #pickle.dump({'accuracy': accuracy, 'report': class_report}, f)

    # End timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    return X_t, y_p, g


In [ ]:
cnn_result = cnn(df_09)

In [ ]:
#convert the returned data into DFs
df0 = cnn_result[0]
df1 = cnn_result[1]
df2 = cnn_result[2]

In [ ]:
#join the 3 DFs
joined_df = df0.join(df1, how='outer').join(df2, how='outer')
joined_df = joined_df.dropna(how='any')
joined_df

**Read Me** we have to manually re-create the DF otherwise the columns would be merged with NaN values in different columns because of indexing. Cleaner this way. Also, There are a lot of neutral and mixed sentences (not of col_type exclusively F or M) so when we include the gender element we lose A LOT OF DATA. We should discuss this, as the gender column is important for the analysis. 

**Table Comparing Model Results**

In [ ]:
results = {' ': ['LSTM', 'CNN'],
           'Accuracy': [0.638, 0.926],
           'Mean Squared Error': [0.193, 0.0605],
           'Test loss': [0.65, 0.223]}

# Create a pandas dataframe from the dictionary
df = pd.DataFrame(results)

# Set the index of the dataframe to the Kernel column
df.set_index(' ', inplace=True)

# Display the dataframe
print(df)

In [ ]:
# Import the required libraries - method with NLP Cloud 
!pip install google-cloud-language
from google.cloud import language_v1

# Instantiate the NLP client
client = language_v1.LanguageServiceClient()

# Load the text dataset
data = df_09

# Define a function to perform sentiment analysis using the Google Cloud NLP API
def analyze_sentiment(text):
    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)
    sentiment = client.analyze_sentiment(request={'document': document}).document_sentiment.score
    return sentiment

# Apply the sentiment analysis function to the text data
data['sentiment'] = data['text'].apply(analyze_sentiment)

# Define a threshold for labeling sentiment as positive or negative
threshold = 0.2

# Label the sentiment as positive or negative based on the threshold
data.loc[data['sentiment'] >= threshold, 'sentiment_label'] = 'positive'
data.loc[data['sentiment'] < threshold, 'sentiment_label'] = 'negative'

# Export the labeled data to a new CSV file
data.to_csv('labeled_text_data.csv', index=False)
